In [ ]:
# install ITSxpress and Q2-itsxpress in the environment
conda install -c bioconda itsxpress
pip install q2-itsxpress

In [ ]:
# refresh plugins
qiime dev refresh-cache

In [ ]:
# check to see if the plugins are installed correctly
qiime itsxpress

In [ ]:
# import ITS2 sequences
qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path ITS2 \
--input-format CasavaOneEightSingleLanePerSampleDirFmt \
--output-path demux-paired-end.qza
# output file: demux-paired-end.qza

In [ ]:
qiime demux summarize \
--i-data demux-paired-end.qza \
--o-visualization demux-paired-end.qzv
# output file: demux-paired-end.qzv

In [ ]:
qiime dada2 denoise-paired \
--i-demultiplexed-seqs demux-paired-end.qza \
--p-trim-left-f 0 \
--p-trim-left-r 0 \
--p-trunc-len-f 0 \
--p-trunc-len-r 0 \
--o-representative-sequences BNW-skin-ITS-paired-rep-seqs.qza \
--o-table BNW-skin-ITS-paired-table.qza \
--o-denoising-stats BNW-skin-ITS-paired-stats-dada2.qza

In [ ]:
# create visualisation files for the files above
qiime metadata tabulate \
--m-input-file BNW-skin-ITS-paired-stats-dada2.qza \
--o-visualization ITS-BNW-skin-stats-dada2.qzv
qiime feature-table summarize \
--i-table BNW-skin-ITS-paired-table.qza \
--o-visualization BNW-skin-ITS-paired-table.qzv
qiime feature-table tabulate-seqs \
--i-data BNW-skin-ITS-paired-rep-seqs.qza \
--o-visualization BNW-skin-ITS-paired-rep-seqs.qzv

In [ ]:
qiime feature-table group \
  --i-table BNW-skin-ITS-paired-table.qza \
  --p-axis sample \
  --m-metadata-file BNW-skin-metadata-index.tsv \
  --m-metadata-column SampleID-new \
  --p-mode sum \
  --o-grouped-table BNW-skin-ITS-paired-reindexed-table.qza

In [ ]:
qiime feature-table summarize \
  --i-table BNW-skin-ITS-paired-reindexed-table.qza \
  --m-sample-metadata-file BNW-skin-metadata-ITS2.tsv \
  --o-visualization BNW-skin-ITS-paired-reindexed-table.qzv

# UNITE database

In [ ]:
# download the UNITE QIIME reference sequences and taxonomy files
wget https://files.plutof.ut.ee/public/orig/C5/54/C5547B97AAA979E45F79DC4C8C4B12113389343D7588716B5AD330F8BDB300C9.tgz

In [ ]:
# decompress file
tar xzf C5547B97AAA979E45F79DC4C8C4B12113389343D7588716B5AD330F8BDB300C9.tgz

In [ ]:
# move into developer environment
cd sh_qiime_release_10.05.2021/developer/

In [ ]:
# fix formatting errors that prevent importation of the reference sequences. 
# There are white spaces and lower case letters that need to be converted
awk '/^>/ {print($0)}; /^[^>]/ {print(toupper($0))}' sh_refs_qiime_ver8_99_10.05.2021_dev.fasta | tr -d ' ' > sh_refs_qiime_ver8_99_10.05.2021_dev_uppercase.fasta

In [ ]:
# import UNITE reference sequences
qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path sh_refs_qiime_ver8_99_10.05.2021_dev_uppercase.fasta \
--output-path unite-ver8-seqs_99.qza

In [ ]:
# import taxonomy file
qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-path sh_taxonomy_qiime_ver8_99_10.05.2021_dev.txt \
--output-path unite-ver8-taxonomy_99.qza \
--input-format HeaderlessTSVTaxonomyFormat

In [ ]:
# train the classifier
qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads unite-ver8-seqs_99.qza \
--i-reference-taxonomy unite-ver8-taxonomy_99.qza \
--o-classifier unite-ver8-99-classifier.qza

In [ ]:
# taxonomoc classification
qiime feature-classifier classify-sklearn \
  --i-classifier sh_qiime_release_10.05.2021/developer/unite-ver8-99-classifier.qza \
  --i-reads BNW-skin-ITS-paired-rep-seqs.qza \
  --o-classification BNW-skin-unite-taxonomy-paired.qza

In [ ]:
qiime taxa barplot \
  --i-table BNW-skin-ITS-paired-reindexed-table.qza \
  --i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
  --m-metadata-file BNW-skin-metadata-ITS2.tsv \
  --o-visualization taxa-bar-plots.qzv

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-unite-taxonomy-paired.qza \
--o-visualization BNW-skin-unite-taxonomy-paired.qzv

In [ ]:
# create rarefaction table 
qiime diversity alpha-rarefaction \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv   \
--o-visualization BNW-skin-ITS-paired-table-rarefaction.qzv \
--p-min-depth 500 \
--p-max-depth 50000

# plateaus around 12K reads

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-min-frequency 2198 \
--o-filtered-table BNW-skin-ITS-paired-table-atleast-2198-reads.qza

# Remove toe samples and uncertain severity groupings from the analysis as well as controls

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "bodysite IN ('toe') OR [Severity] IN ('Uncertain/low mange', 'Observational healthy')" \
--p-exclude-ids \
--o-filtered-table BNW-skin-table_Final-sev.qza

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table_Final-sev.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "[sampletype]='Control'" \
--p-exclude-ids \
--o-filtered-table BNW-skin-table_Final.qza

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table BNW-skin-table_Final.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-min-frequency 2198 \
--o-filtered-table BNW-skin-table-at-least-2198-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--o-visualization BNW-skin-table-at-least-2198-reads.qzv 

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--o-visualization BNW-skin-table-at-least-2198-reads-BARPLOT-PER-SAMPLE.qzv

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-SEVERITY.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-table-SEVERITY.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization BNW-skin-table-SEVERITY-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 5538 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--i-phylogeny unite-rooted-tree.qza \
--p-sampling-depth 5538 \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--output-dir BNW-skin-Core-metrics-filtered-table-5538

In [ ]:
for i in BNW-skin-Core-metrics-filtered-table-5538/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata-ITS2.tsv --o-visualization ${i/.qza/.qzv}; done

In [ ]:
qiime diversity alpha \
  --i-table BNW-skin-table-at-least-2198-reads.qza \
  --p-metric chao1 \
  --o-alpha-diversity BNW-skin-Core-metrics-filtered-table-5538/chao1_index.qza

In [ ]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity BNW-skin-Core-metrics-filtered-table-5538/chao1_index.qza \
  --m-metadata-file BNW-skin-metadata-ITS2.tsv \
  --o-visualization BNW-skin-Core-metrics-filtered-table-5538/chao1_index.qzv

# Permanova test for microbial composition differences

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-filtered-table-5538/bray_curtis_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-filtered-table-5538/bray_curtis_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-filtered-table-5538/jaccard_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-filtered-table-5538/jaccard_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

# ANCOM analysis to identify differentially abundant features/OTUS

In [ ]:
#remove low abundance/prevalence features
qiime feature-table filter-features \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--o-filtered-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500.qza \
--o-visualization BNW-skin-table-FILTERED-ANCOM-ms8-mf500.qzv

In [ ]:
qiime composition add-pseudocount \
--i-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500.qza \
--o-composition-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500-pseudo.qza

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500-pseudo.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column wombatid \
--o-visualization BNW-skin-table-FILTERED-ANCOM-ms8-mf500-pseudo-WOMBAT.qzv

# ANCOM collapsed at family level (i.e. not at the feature level)

In [ ]:
qiime taxa collapse \
--i-table BNW-skin-table-at-least-2198-reads.qza  \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--p-level 5 \
--o-collapsed-table BNW-skin-table-FAMILY.qza

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-FAMILY.qza \
--o-filtered-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime composition add-pseudocount \
--i-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500.qza  \
--o-composition-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo.qza 

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column wombatid \
--o-visualization BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo-WOMBAT.qzv

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY.qzv

# Unique feature analysis

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table BNW-skin-table-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-CONFHEALTHYonly.qza \
--o-visualization BNW-skin-table-CONFHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table BNW-skin-table-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-MANGYonly.qza \
--o-visualization BNW-skin-table-MANGYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table BNW-skin-table-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-SEVEREonly.qza \
--o-visualization BNW-skin-table-SEVEREonly.qzv

# Mitigating cross contamination

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "Sample_Or_Control IN ('Negative control')" \
--o-filtered-table BNW-skin-table-NTCs-ONLY.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NTCs-ONLY.qza \
--o-visualization BNW-skin-table-NTCs-ONLY.qzv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NTCs-ONLY.qza \
--output-path BNW-skin-table-NTCs-ONLY.biom

In [ ]:
biom convert -i BNW-skin-table-NTCs-ONLY.biom/feature-table.biom -o BNW-skin-table-NTCs-ONLY.biom/feature-table.tsv --to-tsv

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-unite-taxonomy-paired.qza \
--o-visualization BNW-skin-unite-taxonomy-paired.qzv

In [ ]:
# There seems to be some cross-contamination. 
# Will compare BNW-skin-reindexed-table.qza to BNW-skin-table-NTCs-ONLY.qza
# to identify low abundance microbes as these are most likely contaminants arrising from extraction or sequencing.
# High abundance microbes could be a transfer from true biological samples, and will be retained in further analysis. 

In [ ]:
qiime taxa filter-table \
  --i-table BNW-skin-table_Final.qza \
  --i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
  --p-exclude 4be7f131da15bb42f3c166ed50ae7b0d,647a8e22386347fbb01da9d546a8bed9,cf16872a26d3bb85b08fe424e831d06d,3ecd5ec3492e64cd82d9c5fad26dddd1,b7e134a69de1650ba8c4f2cf810098a4,9572f06e674f7f92daf452c93104b0f7,12716572e731679126402cc0184db81c,6f964475e33afcb4273de64ca19f8fe1,92ae4092631e9e5900eb542e5724634d,21c7c894cf50cb2fa5e6947feacb1b16,c12fdff66e8e19f97460a3c5bce1be64,6455cc78dd89ca7be69ce159df40e00b,0706fb1f121f9c68f7e87a5129c50c6c,03196f284e70c3a38b6d04b80ee0debc \
  --o-filtered-table table-no-contaminants.qza

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table table-no-contaminants.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-min-frequency 2198 \
--o-filtered-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
--o-visualization BNW-skin-table-no-contaminants-at-least-2198-reads.qzv 

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--o-visualization BNW-skin-table-no-cont-at-least-2198-reads-Final_BARPLOT-PER-SAMPLE.qzv

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-no-cont-SEVERITY.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-table-no-cont-SEVERITY.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization BNW-skin-table-no-cont-SEVERITY-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 5538 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
--i-phylogeny unite-rooted-tree.qza \
--p-sampling-depth 5538 \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--output-dir BNW-skin-Core-metrics-NoCont-filtered-table-5538

In [ ]:
for i in BNW-skin-Core-metrics-NoCont-filtered-table-5538/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata-ITS2.tsv --o-visualization ${i/.qza/.qzv}; done

In [ ]:
qiime diversity alpha \
  --i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
  --p-metric chao1 \
  --o-alpha-diversity BNW-skin-Core-metrics-NoCont-filtered-table-5538/chao1_index.qza

In [ ]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity BNW-skin-Core-metrics-NoCont-filtered-table-5538/chao1_index.qza \
  --m-metadata-file BNW-skin-metadata-ITS2.tsv \
  --o-visualization BNW-skin-Core-metrics-NoCont-filtered-table-5538/chao1_index.qzv

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-NoCont-filtered-table-5538/bray_curtis_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-NoCont-filtered-table-5538/bray_curtis_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-NoCont-filtered-table-5538/jaccard_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-NoCont-filtered-table-5538/jaccard_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

# Unique feature analysis of severity groupings without contaminants - split into the three severity groupings

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table BNW-skin-table-NoCont-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly.qza \
--o-visualization BNW-skin-table-NoCont-CONFHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table BNW-skin-table-NoCont-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-MANGYonly.qza \
--o-visualization BNW-skin-table-NoCont-MANGYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-contaminants-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table BNW-skin-table-NoCont-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-SEVEREonly.qza \
--o-visualization BNW-skin-table-NoCont-SEVEREonly.qzv

# Provide overview of ASVs for each severity type

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NoCont-CONFHEALTHYonly.qza \
--output-path BNW-skin-table-NoCont-CONFHEALTHYonly.biom

In [ ]:
biom convert -i BNW-skin-table-NoCont-CONFHEALTHYonly.biom/feature-table.biom -o BNW-skin-table-NoCont-CONFHEALTHYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NoCont-MANGYonly.qza \
--output-path BNW-skin-table-NoCont-MANGYonly.biom

In [ ]:
biom convert -i BNW-skin-table-NoCont-MANGYonly.biom/feature-table.biom -o BNW-skin-table-NoCont-MANGYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NoCont-SEVEREonly.qza \
--output-path BNW-skin-table-NoCont-SEVEREonly.biom

In [ ]:
biom convert -i BNW-skin-table-NoCont-SEVEREonly.biom/feature-table.biom -o BNW-skin-table-NoCont-SEVEREonly.biom/feature-table.tsv --to-tsv

# Unique ASVs - min 3 samples due to severity groupings

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly.qza \
--o-filtered-table BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qza \
--p-min-samples 3 

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qza \
--o-visualization BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qzv

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-NoCont-MANGYonly.qza \
--o-filtered-table BNW-skin-table-NoCont-MANGYonly-ms3.qza \
--p-min-samples 3

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-MANGYonly-ms3.qza \
--o-visualization BNW-skin-table-NoCont-MANGYonly-ms3.qzv

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-NoCont-SEVEREonly.qza \
--o-filtered-table BNW-skin-table-NoCont-SEVEREonly-ms3.qza \
--p-min-samples 3 

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-SEVEREonly-ms3.qza \
--o-visualization BNW-skin-table-NoCont-SEVEREonly-ms3.qzv

# Merge groups for taxa barplot

In [ ]:
qiime feature-table group \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-NoCont-CONFHEALTHY-ms3-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table BNW-skin-table-NoCont-MANGYonly-ms3.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-NoCont-MANGY-ms3-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table BNW-skin-table-NoCont-SEVEREonly-ms3.qza \
--m-metadata-file BNW-skin-metadata-ITS2.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-NoCont-SEVERE-ms3-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table merge \
--i-tables BNW-skin-table-NoCont-CONFHEALTHY-ms3-wombat-COLLAPSED.qza BNW-skin-table-NoCont-MANGY-ms3-wombat-COLLAPSED.qza BNW-skin-table-NoCont-SEVERE-ms3-wombat-COLLAPSED.qza \
--o-merged-table BNW-skin-table-NoCont-wombat-COLLASPED.qza

# Technical replicate survey